In [19]:
import sys
sys.path.append("./ControlNet-bk") 

In [21]:
from ControlNet_bk.gradio_hed2image import  process
import json
import sagemaker

Running on local URL:  http://0.0.0.0:7861
Running on public URL: https://91f2a0ffe3c2341381.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


[array([[[ 91, 110, 113],
        [ 85, 107, 116],
        [ 76,  97, 107],
        ...,
        [ 91, 103, 106],
        [ 91, 106, 109],
        [119, 127, 130]],

       [[ 84, 103, 113],
        [ 88, 110, 120],
        [ 79, 100, 110],
        ...,
        [ 94, 102, 105],
        [ 91, 102, 107],
        [118, 126, 136]],

       [[ 84, 104, 114],
        [ 86, 107, 119],
        [ 80, 103, 114],
        ...,
        [ 93, 103, 107],
        [ 93, 105, 108],
        [120, 130, 132]],

       ...,

       [[ 54,  52,  66],
        [ 53,  51,  71],
        [ 45,  44,  66],
        ...,
        [ 54,  61,  94],
        [ 55,  63,  95],
        [ 52,  56,  84]],

       [[ 59,  55,  68],
        [ 55,  51,  71],
        [ 45,  43,  65],
        ...,
        [ 56,  58,  94],
        [ 56,  64,  95],
        [ 58,  63,  93]],

       [[ 48,  43,  59],
        [ 47,  45,  65],
        [ 42,  42,  62],
        ...,
        [ 47,  52,  87],
        [ 51,  59,  91],
        [ 59,  57,  84]

In [ ]:
results = process()

In [11]:
from share import *
import config

import cv2
import einops
import gradio as gr
import numpy as np
# import torch
import random
import base64
import json
import sys
import io, os
import signal
import traceback
# import flask

# from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.hed import HEDdetector
# from cldm.model import create_model, load_state_dict
# from cldm.ddim_hacked import DDIMSampler

In [12]:
### hardcode values
prompt = "oil painting"
num_samples = 1
image_resolution = 512
strength = 1.0
guess_mode = False
detect_resolution = 512
ddim_steps = 20
scale = 9.0
seed = 1213180976
eta = 0.0
a_prompt = 'best quality, extremely detailed'
n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'

############

In [14]:
apply_hed = HEDdetector()
from PIL import Image
input_image = np.array(Image.open("old.png").convert("RGB")).astype(np.uint8)

input_image = HWC3(input_image)
detected_map = apply_hed(resize_image(input_image, detect_resolution))
detected_map = HWC3(detected_map)

img = resize_image(input_image, image_resolution)
H, W, C = img.shape

detected_map = cv2.resize(detected_map, (W, H), interpolation=cv2.INTER_LINEAR) 

data = {
    "detected_map": base64.b64encode(detected_map).decode("utf-8"), 
    "prompt": prompt, 
    "a_prompt": a_prompt, 
    "n_prompt": n_prompt, 
    "num_samples": num_samples,  
    "ddim_steps": ddim_steps, 
    "guess_mode": guess_mode, 
    "strength": strength, 
    "scale": scale, 
    "seed": seed, 
    "eta": eta, 
    "H": H, 
    "W": W, 
    "C": C
       }

In [15]:
import io, json
with io.open('data.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(data))

In [6]:
!pwd

/home/ec2-user/SageMaker/ControlNet_SageMaker_Hosting/container


In [4]:
import tarfile
from pathlib import Path
import tqdm
import sagemaker
from sagemaker.s3 import S3Uploader, s3_path_join

In [5]:
def create_tar(tarfile_name: str, local_path: Path):
    """
    Create a tar.gz archive with the content of `local_path`.
    """
    file_list = [k for k in local_path.glob("**/*.*") if f"{k.relative_to(local_path)}"[0] != "."]
    pbar = tqdm.tqdm(file_list, unit="files")
    with tarfile.open(tarfile_name, mode="w:gz") as archive:
        for k in pbar:
            pbar.set_description(f"{k}")
            archive.add(k, arcname=f"{k.relative_to(local_path)}")
    tar_size = Path(tarfile_name).stat().st_size / 10**6
    return 

In [8]:
%%time
tar_file = "model.tar.gz"
tar_size = create_tar(tar_file, Path("./ControlNet/models"))
print(f"Created {tar_file}")

ControlNet/models/control_sd15_hed.pth: 100%|██████████| 3/3 [05:47<00:00, 115.90s/files]

Created model.tar.gz, size None MB
CPU times: user 5min 43s, sys: 4.67 s, total: 5min 48s
Wall time: 5min 47s


In [9]:
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = "sagemaker/generative-ai-controlnet"

In [10]:
model_data_path = s3_path_join("s3://", bucket, prefix + "/models")
print(f"Uploading Models to {model_data_path}")
model_uri = S3Uploader.upload("model.tar.gz", model_data_path)
print(f"Uploaded roberta model to {model_uri}")

Uploading Models to s3://sagemaker-us-east-1-631450739534/sagemaker/generative-ai-controlnet/models
Uploaded roberta model to s3://sagemaker-us-east-1-631450739534/sagemaker/generative-ai-controlnet/models/model.tar.gz
